<a href="https://colab.research.google.com/github/Sonica-B/Deep-Learning-CS541_F2024/blob/HomeWork4/homework4(3)_Shreya_Boyane_Ankit_Gole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import copy


In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize(224),            # Resize images to 224x224 pixels
    transforms.Grayscale(3),           # Convert grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Load the dataset
train_dataset = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='data', train=False, transform=transform, download=True)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 26.4M/26.4M [00:03<00:00, 8.45MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 175kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.15MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 19.8MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
# Load pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final layer for 10 classes (Fashion MNIST)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # 10 classes in Fashion MNIST


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]


In [ ]:
# Fine-tune the entire model
for param in model.parameters():
    param.requires_grad = True


# Train only the final layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001 if model.fc.requires_grad else 0.0001)

def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Unfreeze all layers for fine-tuning
for param in model.parameters():
    param.requires_grad = True

# Define the optimizer (do this after setting requires_grad=True)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# Train Only Final Layer
print("Training Only Final Layer")
train_model(model, train_loader, criterion, optimizer, epochs=5)
final_layer_accuracy = evaluate_model(model, test_loader)
print(f"Final Layer Only Accuracy: {final_layer_accuracy}%")


Training Only Final Layer
Epoch 1/5, Loss: 0.3680
Epoch 2/5, Loss: 0.1947
Epoch 3/5, Loss: 0.1535
Epoch 4/5, Loss: 0.1231
Epoch 5/5, Loss: 0.0975
Test Accuracy: 93.36%
Final Layer Only Accuracy: 93.36%


In [ ]:
# Fine-Tune the Whole Model
print("Fine-Tuning the Entire Model")
train_model(model, train_loader, criterion, optimizer, epochs=5)
fine_tune_accuracy = evaluate_model(model, test_loader)
print(f"Fine-Tune Accuracy: {fine_tune_accuracy}%")

Fine-Tuning the Entire Model
Epoch 1/5, Loss: 0.0742
Epoch 2/5, Loss: 0.0558
Epoch 3/5, Loss: 0.0384
Epoch 4/5, Loss: 0.0284
Epoch 5/5, Loss: 0.0206
Test Accuracy: 93.96%
Fine-Tune Accuracy: 93.96%
